# Tutorial: Electrothermal KPI Gate Consistency

**Audience:** contributors maintaining CI benchmark gates.

**You will learn:**

- how component-level electrothermal KPI metrics are evaluated,
- how `kpi_gate.run_gate(...)` passes/fails based on consistency thresholds,
- how to build a minimal reproducible gate fixture.


## Outline

1. Import `benchmarks/kpi_gate.py` as a module.
2. Build a synthetic benchmark result payload.
3. Run gate with a passing telemetry case.
4. Run gate with a failing component-consistency case.


In [ ]:
# Setup cell: keep it short and deterministic
from __future__ import annotations

import hashlib
import json
import tempfile
from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    here = start.resolve()
    for candidate in (here, *here.parents):
        if (candidate / "pyproject.toml").is_file() and (candidate / "benchmarks").is_dir():
            return candidate
    raise RuntimeError("Could not locate repository root")

repo_root = find_repo_root(Path.cwd())
benchmarks_dir = repo_root / "benchmarks"
if str(benchmarks_dir) not in sys.path:
    sys.path.insert(0, str(benchmarks_dir))

import kpi_gate


## Step 1 - Build pass/fail gate fixtures

This example writes temporary baseline/manifest/threshold files and runs `kpi_gate.run_gate` twice.

Expected:

- pass case: `overall_status == "passed"`
- fail case: `overall_status == "failed"` with component consistency metric failures


In [ ]:
# Minimal working example
with tempfile.TemporaryDirectory() as td:
    root = Path(td)
    bench_path = root / "bench_results.json"
    baseline_path = root / "kpi_baseline.json"
    manifest_path = root / "artifact_manifest.json"
    thresholds_path = root / "thresholds.json"

    pass_telemetry = {
        "component_coverage_rate": 1.0,
        "component_coverage_gap": 0.0,
        "component_loss_summary_consistency_error": 0.0,
        "component_thermal_summary_consistency_error": 0.0,
    }

    bench_payload = {
        "results": [
            {
                "benchmark_id": "buck_electrothermal",
                "scenario": "fixed_mode",
                "status": "passed",
                "runtime_s": 0.10,
                "telemetry": pass_telemetry,
            }
        ]
    }
    bench_path.write_text(json.dumps(bench_payload), encoding="utf-8")

    baseline_payload = {
        "schema_version": "pulsim-kpi-baseline-v1",
        "baseline_id": "tutorial_baseline",
        "captured_at_utc": "2026-02-25T00:00:00Z",
        "source_bench_results": str(bench_path.resolve()),
        "metrics": dict(pass_telemetry),
        "environment": {
            "os": "tutorial-os",
            "python": "3.13",
            "machine_class": "ci",
            "compiler": "clang",
            "cc": "clang",
            "cmake": "cmake",
            "cxx_flags": "-O3",
        },
    }
    baseline_path.write_text(json.dumps(baseline_payload), encoding="utf-8")

    digest = hashlib.sha256(bench_path.read_bytes()).hexdigest()
    manifest_payload = {
        "schema_version": "pulsim-kpi-baseline-manifest-v1",
        "baseline_id": "tutorial_baseline",
        "captured_at_utc": "2026-02-25T00:00:00Z",
        "files": [{"path": str(bench_path.resolve()), "sha256": digest, "size_bytes": bench_path.stat().st_size}],
    }
    manifest_path.write_text(json.dumps(manifest_payload), encoding="utf-8")

    thresholds_payload = {
        "schema": "pulsim-kpi-gates-v1",
        "version": 1,
        "metrics": {
            "component_coverage_rate": {"direction": "higher_is_better", "max_regression_abs": 0.01, "required": True},
            "component_coverage_gap": {"direction": "lower_is_better", "max_regression_abs": 0.0, "required": True},
            "component_loss_summary_consistency_error": {"direction": "lower_is_better", "max_regression_abs": 1e-6, "required": True},
            "component_thermal_summary_consistency_error": {"direction": "lower_is_better", "max_regression_abs": 1e-6, "required": True},
        },
    }
    thresholds_path.write_text(json.dumps(thresholds_payload), encoding="utf-8")

    pass_report = kpi_gate.run_gate(
        baseline_path=baseline_path,
        thresholds_path=thresholds_path,
        bench_results_path=bench_path,
        parity_ltspice_results_path=None,
        parity_ngspice_results_path=None,
        stress_summary_path=None,
    )
    print("PASS run ->", pass_report["overall_status"])

    fail_payload = json.loads(bench_path.read_text(encoding="utf-8"))
    fail_payload["results"][0]["telemetry"].update(
        {
            "component_coverage_rate": 0.75,
            "component_coverage_gap": 1.0,
            "component_loss_summary_consistency_error": 2.0e-2,
            "component_thermal_summary_consistency_error": 1.0e-2,
        }
    )
    bench_path.write_text(json.dumps(fail_payload), encoding="utf-8")

    fail_digest = hashlib.sha256(bench_path.read_bytes()).hexdigest()
    manifest_payload["files"][0]["sha256"] = fail_digest
    manifest_payload["files"][0]["size_bytes"] = bench_path.stat().st_size
    manifest_path.write_text(json.dumps(manifest_payload), encoding="utf-8")

    fail_report = kpi_gate.run_gate(
        baseline_path=baseline_path,
        thresholds_path=thresholds_path,
        bench_results_path=bench_path,
        parity_ltspice_results_path=None,
        parity_ngspice_results_path=None,
        stress_summary_path=None,
    )
    print("FAIL run ->", fail_report["overall_status"])
    print("failed_required_metrics ->", fail_report["failed_required_metrics"])
    print("comparison keys ->", sorted(fail_report["comparisons"].keys()))

    assert pass_report["overall_status"] == "passed"
    assert fail_report["overall_status"] == "failed"
    assert fail_report["failed_required_metrics"] >= 1


## Exercises

1. Tighten `max_regression_abs` for `component_coverage_rate` and observe sensitivity.
2. Set one metric as `required: false` and confirm status becomes `skipped`/non-blocking.
3. Corrupt manifest hash and compare strict vs non-strict provenance mode.


In [ ]:
# Exercise answer scaffold
# Reuse the temporary fixture above and adjust thresholds/telemetry.
# Then inspect fail_report["comparisons"] and fail_report["provenance"].

# TODO: implement your experiment here.
